## Wczytanie Bibliotek

In [ ]:
from PIL import Image
import sys
import numpy as np
import cv2
import tifffile
import plotly.express as px
from IPython import display
from matplotlib import pyplot as plt
import pandas as pd
from timeit import timeit
from memory_profiler import profile
import random
from IPython.display import IFrame
import ipywidgets as widgets
import plotly
from sklearn import preprocessing
import plotly.graph_objs as go
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)
import czifile
from skimage import exposure, img_as_ubyte
from sklearn_extra.cluster import KMedoids

In [ ]:
img = czifile.imread('dataset/rgb/Centr_tel Brachy/Experiment-392.czi')

In [ ]:
img.shape

In [ ]:
img_3channel = img[0,0,:,:,:,0]

In [ ]:
DAPI = (img_3channel[0,:,:])

In [ ]:
plt.imshow(DAPI, cmap='gray')

In [ ]:
DAPI

In [ ]:
DAPI_8bit_c = (DAPI / DAPI.max()) * 255.
DAPI_8bit_c = np.uint8(DAPI_8bit_c)
plt.imshow(DAPI_8bit_c, cmap='gray')

In [ ]:
DAPI_8bit_c

In [ ]:
#Normalize then scale to 255 and convert to uint8 - using opencv
DAPI_8bit_d = cv2.normalize(DAPI, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
plt.imshow(DAPI_8bit_d, cmap='gray')

In [ ]:
DAPI_8bit_d

In [ ]:
#Normalize then scale to 255 and convert to uint8 - using skimage
from skimage import exposure, img_as_ubyte
#exposure.rescale_intensity stretches (scales) intensity to min and max 
# of the dtype of the image. In our case: 0 to 65535
DAPI_8bit_e = img_as_ubyte(exposure.rescale_intensity(DAPI))
plt.imshow(DAPI_8bit_e, cmap='gray')

In [ ]:
DAPI_8bit_e

## Wczytanie Plików 

In [ ]:
# wczytanie ścieżki do plików exp392 
file_red = 'dataset/rgb/Centr_tel Brachy/Experiment-392.czi - C=r.tif'
file_green = 'dataset/rgb/Centr_tel Brachy/Experiment-392.czi - C=g.tif'
file_blue = 'dataset/rgb/Centr_tel Brachy/Experiment-392.czi - C=b.tif'

tif_red = tifffile.imread(file_red)
tif_green = tifffile.imread(file_green)
tif_blue = tifffile.imread(file_blue)

In [ ]:
# reducing z (layers)
tif_red=np.delete(tif_red, np.s_[5:],axis=0)
tif_green=np.delete(tif_green, np.s_[5:],axis=0)
tif_blue=np.delete(tif_blue, np.s_[5:],axis=0)

# tif_red=np.delete(tif_red, np.s_[500:],axis=1)
# tif_green=np.delete(tif_green, np.s_[500:],axis=1)
# tif_blue=np.delete(tif_blue, np.s_[500:],axis=1)

# tif_red=np.delete(tif_red, np.s_[500:],axis=2)
# tif_green=np.delete(tif_green, np.s_[500:],axis=2)
# tif_blue=np.delete(tif_blue, np.s_[500:],axis=2)

## Normalizacja

In [ ]:
tif_red_norm = np.empty(shape=(18, 1040,1388), dtype='uint8')
tif_green_norm = np.empty(shape=(18, 1040,1388), dtype='uint8')
tif_blue_norm = np.empty(shape=(18, 1040,1388), dtype='uint8')

In [ ]:
for i in range(0,tif_blue.shape[0]):
    tif_red_norm[i] = img_as_ubyte(exposure.rescale_intensity(tif_red[i]))
    tif_green_norm[i] = img_as_ubyte(exposure.rescale_intensity(tif_green[i]))
    tif_blue_norm[i] = img_as_ubyte(exposure.rescale_intensity(tif_blue[i]))   

In [ ]:
plt.imshow(tif_blue_norm[0], cmap='Blues')

## Spłaszczenie

In [ ]:
tif_red_flat = tif_red_norm.flatten()
tif_green_flat = tif_green_norm.flatten()
tif_blue_flat = tif_blue_norm.flatten()

 ## 3d array na 2d array
 Konwersja 3d array (jednokolorowy plik tiff) na 2d array zawierający współrzędne pixeli

In [ ]:
# new method for changing 3d array to 2d array with x, y, z, color and size as values


z, y, x = tif_red.shape

x_size = x
y_size = y
z_size = z

z_list = []
for i in range(z_size):
    z_list += [i] * (y_size * x_size)

y_list = []
for i in range(y_size):
    y_list += [i] * (x_size)
y_list *= z_size

x_list = []
x_list = np.arange(0, x_size).tolist() * (z_size * y_size)

# connecting lists into 2d numpy array
tif_arr = np.c_[z_list, y_list, x_list, tif_red_flat, tif_green_flat, tif_blue_flat, np.ones((z_size * y_size* x_size),dtype='int32')]

tif_arr.shape

In [ ]:
z_list

In [ ]:
colors_arr = tif_arr[:,3]
colors_arr = [f'rgb({"{},0,0".format(color)})' for color in colors_arr]

In [ ]:
colors_arr

## Czyszczenie 

In [ ]:
print(tif_arr.shape)
tif_arr_cls = tif_arr[(tif_arr[:,3]>100)|(tif_arr[:,4]>100)|(tif_arr[:,5]>100)]
print(tif_arr_cls.shape)

## Metoda Centroidów - kolorowy tiff

In [ ]:
def centeroidnp(arr):
    length = arr.shape[0]
    sum_z = np.sum(arr[:, 0])
    sum_y = np.sum(arr[:, 1])
    sum_x = np.sum(arr[:, 2])
    sum_r = np.sum(arr[:, 3])
    sum_g = np.sum(arr[:, 4])
    sum_b = np.sum(arr[:, 5])
    return sum_z/length, sum_y/length, sum_x/length, sum_r/length, sum_g/length, sum_b/length, length

In [ ]:
# defining empty 2d array where centroids will be stored
@widgets.interact(radius=(1,10), zradius=(1,3), color_diff=(5,50,5),smin=(0,25,5),smax=(20,60,5), start=(False,True))
def centroidMethod(start,radius, zradius, color_diff, smin, smax,):
    if start:
        centroids = np.empty([0,7])

        # duplicating f1_arr of pixels
        dup = tif_arr_cls

        # looping thorugh duplicated array until it's empty
        while len(dup)>0:

            # assign starting point to random pixel
            pixel = random.choice(dup)

            # creating matrix of neighbours
            neighbours_arr = dup[((dup[:,0]<=pixel[0]+zradius) 
                                & (dup[:,0]>=pixel[0]-zradius)
                                & (dup[:,1]<=pixel[1]+radius)
                                & (dup[:,1]>=pixel[1]-radius)
                                & (dup[:,2]<=pixel[2]+radius)
                                & (dup[:,2]>=pixel[2]-radius)
                                & (dup[:,3]<=pixel[3]+color_diff)
                                & (dup[:,3]>=pixel[3]-color_diff)
                                & (dup[:,4]<=pixel[4]+color_diff)
                                & (dup[:,4]>=pixel[4]-color_diff)
                                & (dup[:,5]<=pixel[5]+color_diff)
                                & (dup[:,5]>=pixel[5]-color_diff))]

            # calculating centroid vector from array of neighbours
            centroid = centeroidnp(neighbours_arr)

            # adding centroid to centroids matrix
            centroids=np.vstack([centroids, centroid])

            # deleting neighbours from duplicated array
            dims = np.maximum(neighbours_arr.max(0),dup.max(0))+1
            dup = dup[~np.in1d(np.ravel_multi_index(dup.T,dims),np.ravel_multi_index(neighbours_arr.T,dims))]

        colors_arr = centroids[:,3:6]

        colors_arr = [f'rgb({"{},{},{}".format(color[0],color[1],color[2])})' for color in colors_arr]

        trace1 = go.Scatter3d(
            x=centroids[:,2], 
            y=centroids[:,1], 
            z=centroids[:,0],
            mode="markers",
            marker_color=colors_arr,
            marker=dict(
                sizemode="area",
                sizeref=centroids[:,6].max() / smax ** 2,
#                 sizemin=smin,
                opacity=1,
                size=centroids[:,6],
                line=dict(width=0)
            )
        )

        data = [trace1]
        title = "Tiff 3D"
        layout = go.Layout(height=800, width=1000, title=title)

        fig = go.Figure(data=data, layout=layout)

        fig.show()     

In [ ]:
centroids.shape

In [ ]:
colors_arr = centroids[:,3:6]

colors_arr = [f'rgb({"{},{},{}".format(color[0],color[1],color[2])})' for color in colors_arr]

trace1 = go.Scatter3d(
    x=centroids[:,2], 
    y=centroids[:,1], 
    z=centroids[:,0],
    mode="markers",
    marker_color=colors_arr,
    marker=dict(
        sizemode="area",
        sizeref=centroids[:,6].max() / 60 ** 2,
        sizemin=20,

        size=centroids[:,6],
        line=dict(width=0)

    )

)

data = [trace1]
title = "Tiff 3D"
layout = go.Layout(height=800, width=1000, title=title)

fig = go.Figure(data=data, layout=layout)

fig.show()     

In [ ]:
test=tif_arr_cls[20:30]

In [ ]:
test

In [ ]:
test

In [ ]:
for i in range(0,10):
    test[i][6]=50

In [ ]:
test[1][6]=1
test[5][6]=10


In [ ]:
test

In [ ]:
colors_arr = test[:,3:6]

colors_arr = [f'rgb({"{},{},{}".format(color[0],color[1],color[2])})' for color in colors_arr]

trace1 = go.Scatter3d(
    x=test[:,2], 
    y=test[:,1], 
    z=test[:,0],
    mode="markers",
    marker_color=colors_arr,
    marker=dict(
        sizemode="area",
#         sizeref=test[:,6].max() / 60 ** 2,
#         sizemin=20,

#         size=test[:,6],
        line=dict(width=0)

    )

)

data = [trace1]
title = "Tiff 3D"
layout = go.Layout(height=1000, width=1000, title=title)

fig = go.Figure(data=data, layout=layout)

fig.show()     

In [ ]:
colors_arr = test[:,3:6]

colors_arr = [f'rgb({"{},{},{}".format(color[0],color[1],color[2])})' for color in colors_arr]

trace1 = go.Scatter3d(
    x=test[:,2], 
    y=test[:,1], 
    z=test[:,0],
    mode="markers",
    marker_color=colors_arr,
    marker=dict(
        sizemode="area",
        sizeref=test[:,6].max() / 50 ** 2,
#         sizemin=20,
        size=test[:,6],         opacity=1,
        line=dict(width=0)

    )

)

data = [trace1]
title = "Tiff 3D"
layout = go.Layout(height=1000, width=1000, title=title)

fig = go.Figure(data=data, layout=layout)

fig.show()     

In [ ]:
test[:,6]

### k-medoids

In [ ]:
img26_file = 'dataset/rgb/FISH_Brachy/Image0026-rgb.tif' 

# adding z dimention with tiffile library
img26 = tifffile.imread(img26_file)

In [ ]:
img26=np.delete(img26, np.s_[:7],axis=0)
img26=np.delete(img26, np.s_[1:],axis=0)

In [ ]:
img26.shape

In [ ]:
img26_flat = img26.flatten()

z, y, x, rgb = img26.shape

x_size = x
y_size = y
z_size = z

z_list = []
for i in range(z_size):
    z_list += [i] * (y_size * x_size)

y_list = []
for i in range(y_size):
    y_list += [i] * (x_size)
y_list *= z_size

x_list = []
x_list = np.arange(0, x_size).tolist() * (z_size * y_size)

# połączenie list w 2d numpy array
img26_arr = np.c_[z_list, y_list, x_list, img26_flat[::3], img26_flat[1::3], img26_flat[2::3], np.ones((z_size * y_size* x_size),dtype=int)]

img26_arr.shape

In [ ]:
img26_clr = img26_arr[(img26_arr[:,3]>100)|(img26_arr[:,4]>100)|(img26_arr[:,5]>100)]
img26_clr.shape

In [ ]:
# defining empty 2d array where centroids will be stored
@widgets.interact(radius=(1,10), zradius=(1,3), color_diff=(5,50,5),smin=(0,25,5),smax=(20,60,5), start=(False,True))
def centroidMethod(start,radius, zradius, color_diff, smin, smax,):
    if start:
        medoids = np.empty([0,7])

        # duplicating f1_arr of pixels
        dup = img26_clr

        # looping thorugh duplicated array until it's empty
        while len(dup)>0:

            # assign starting point to random pixel
            pixel = random.choice(dup)

            # creating matrix of neighbours
            neighbours_arr = dup[((dup[:,0]<=pixel[0]+zradius) 
                                & (dup[:,0]>=pixel[0]-zradius)
                                & (dup[:,1]<=pixel[1]+radius)
                                & (dup[:,1]>=pixel[1]-radius)
                                & (dup[:,2]<=pixel[2]+radius)
                                & (dup[:,2]>=pixel[2]-radius)
                                & (dup[:,3]<=pixel[3]+color_diff)
                                & (dup[:,3]>=pixel[3]-color_diff)
                                & (dup[:,4]<=pixel[4]+color_diff)
                                & (dup[:,4]>=pixel[4]-color_diff)
                                & (dup[:,5]<=pixel[5]+color_diff)
                                & (dup[:,5]>=pixel[5]-color_diff))]
            
            
            idx_m = KMedoids(n_clusters=1, random_state=0).fit(neighbours_arr[:,3:6]).medoid_indices_
            # calculating centroid vector from array of neighbours
            
            medoid = neighbours_arr[idx_m]
            
            medoid[0,6]=neighbours_arr.shape[0]

            # adding centroid to centroids matrix
            medoids=np.vstack([medoids, medoid])

            # deleting neighbours from duplicated array
            dims = np.maximum(neighbours_arr.max(0),dup.max(0))+1
            dup = dup[~np.in1d(np.ravel_multi_index(dup.T,dims),np.ravel_multi_index(neighbours_arr.T,dims))]

        colors_arr = medoids[:,3:6]

        colors_arr = [f'rgb({"{},{},{}".format(color[0],color[1],color[2])})' for color in colors_arr]

        trace1 = go.Scatter3d(
            x=medoids[:,2], 
            y=medoids[:,1], 
            z=medoids[:,0],
            text =medoids[:,6],
            mode="markers",
            marker_color=colors_arr,
            marker=dict(
                sizemode="area",
                sizeref=medoids[:,6].max() / smax ** 2,
#                 sizemin=smin,
                opacity=1,
                size=medoids[:,6],
                line=dict(width=0)
            )
        )

        data = [trace1]
        title = "Tiff 3D"
        layout = go.Layout(height=800, width=1000, title=title)

        fig = go.Figure(data=data, layout=layout)

        fig.show()     

In [ ]:
b = np.random.randint(0, 5, [3, 3])

In [ ]:
plt.hist(img26_clr[:,5], bins=100, range=(0,255));

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

# Set the figure size
plt.rcParams["figure.figsize"] = [7.50, 3.50]
plt.rcParams["figure.autolayout"] = True

colors_arr = img26_clr[:,3:6]

colors_arr = [f'rgb({"{},{},{}".format(color[0],color[1],color[2])})' for color in colors_arr]

# Random data of 100×3 dimension
data = np.array(np.random.random((100, 3)))

# Scatter plot
plt.scatter(x=img26_clr[:,2], y=img26_clr[:,1], c=img26_clr[:,3])

# Display the plot
plt.show()

In [ ]:
t=img26_clr[:,3:6]

In [ ]:
t[[0]].shape

In [ ]:
t[[0]]

In [ ]:
from sklearn.cluster import DBSCAN
clustering = DBSCAN(eps=1,min_samples=5).fit(img26_clr[:,:4])
cluster = clustering.labels_

In [ ]:
len(set(cluster))

In [ ]:
df = DataFrame(dict(x=img26_clr[:,2],y=img26_clr[:,1], label=cluster))
c